In [18]:
import pandas as pd

In [19]:
!ls

Project_CE.docx     TDCE.csv            version_space.ipynb


In [20]:
df = pd.read_csv("TDCE.csv")

In [21]:
df.head()

,Sky,AirTemp,Humidity,Wind,Water,Forecast,class
0,Rainy,Cold,Normal,Weak,Cool,Same,N
1,Rainy,Cold,Normal,Weak,Warm,Same,N
2,Sunny,Warm,Normal,Weak,Warm,Same,Y
3,Sunny,Warm,Normal,Weak,Warm,Same,Y
4,Rainy,Cold,High,Weak,Cool,Change,N


In [22]:
df.shape

(1000, 7)

In [23]:
for col in df.columns:
    print(f"{col} -> {df[col].unique()} -> {df[col].nunique()}")

Sky -> ['Rainy' 'Sunny' 'Cloudy'] -> 3
AirTemp -> ['Cold' 'Warm'] -> 2
Humidity -> ['Normal' 'High'] -> 2
Wind -> ['Weak' 'Strong'] -> 2
Water -> ['Cool' 'Warm'] -> 2
Forecast -> ['Same' 'Change'] -> 2
class -> ['N' 'Y'] -> 2


In [24]:
df["class"] = df["class"].replace({"Y": 1, "N": 0})
for col in df.columns:
    print(f"{col} -> {df[col].unique()} -> {df[col].nunique()}")

Sky -> ['Rainy' 'Sunny' 'Cloudy'] -> 3
AirTemp -> ['Cold' 'Warm'] -> 2
Humidity -> ['Normal' 'High'] -> 2
Wind -> ['Weak' 'Strong'] -> 2
Water -> ['Cool' 'Warm'] -> 2
Forecast -> ['Same' 'Change'] -> 2
class -> [0 1] -> 2


In [25]:
from __future__ import annotations

class WhatHappenedError(Exception):
    pass

class Hypothesis(list):
    def is_consistent_with(self, d: pd.Series) -> bool:
        if len(d) != len(self):
            raise LenMismatchError(len(self), len(d))
            
        for (_h, _d) in zip(self, d):
            if not is_more_general_than_or_equal(_h, _d):
                return False

        return True

    def predict(self, d: pd.Series) -> bool:
        return self.is_consistent_with(d)

    def is_more_general_than(self, h2: Hypothesis) -> bool:
        if len(h2) != len(self):
            raise LenMismatchError(len(self), len(h2))

        for _h1, _h2 in zip(self, h2):
            if not is_more_general_than(_h1, _h2):
                return False

        return True

    def is_more_specific_than(self, h2: Hypothesis) -> bool:
        if len(d) != len(self):
            raise LenMismatchError(len(self), len(d))

        for _h1, _h2 in zip(self, h2):
            if is_more_general_or_equal(_h1, _h2):
                return False

        return True

def is_more_general_than_or_equal(_h1, _h2) -> bool:
    if _h1 == ANY:
        return True
        
    if _h1 == _h2:
        return True

    return False

def is_more_general_than(_h1, _h2) -> bool:
    if _h2 == NULL:
        if _h1 != NULL:
            return True
        return False

    if _h2 == ANY:
        return False

    if _h1 == _h2:
        return False

    raise WhatHappenedError
    
class InvalidClassError(Exception):
    pass

NULL = "0"
ANY = "?"

In [26]:
h = Hypothesis(["b", "b", ANY])
d = pd.Series(["a", "b", "c"])
h.predict(d)

False

<hr>

<b>Candidate Elimination Algorithm:</b>


- Initialize $G$ to the set of maximally general hypotheses in H
- Initialize $S$ to the set of maximally specific hypothesis in H
For each training example $d$, do:
    - If $d$ is a positive example:
        - Remove from $G$ any hypothesis inconsistent with $d$
        - For each hypothesis $s$ in $S$ that is not consistent with $d$:
            - Remove $s$ from $S$
            - Add to $S$ all minimal generalizations $h$ of $s$ such that: $h$ is consistent with $d$, and some member of $G$ is more general than $h$
            - Remove from $S$ any hypothesis that is more general than another hypothesis in $S$
    - If $d$ is a negative example:
        - Remove from $S$ any hypothesis inconsistent with $d$
        - For each hypothesis $g$ in $G$ that is not consistent with $d$:
            - Remove $g$ from $G$
            - Add to $G$ all minimal specializations $h$ of $g$ such that: $h$ is consistent with $d$, and some member of $S$ is more specific than $h$
            - Remove from $G$ any hypothesis that is less general than another hypothesis in $G$

In [27]:
class CandidateElimination:
    def __init__(self) -> None:
        pass   

    def fit(self, D: pd.DataFrame, Y: pd.Series, pos: int = 1, neg: int = 0) -> None:
        self.D = D
        self.Y = Y
        self.POS = pos
        self.NEG = neg
        self.G: List[Hypothesis] = [Hypothesis([ANY for _ in self.D.columns])]
        self.S: List[Hypothesis] = [Hypothesis([NULL for _ in self.D.columns])]

        print("initial G:", self.G)
        print("initial S:", self.S)
        
        for (i, d), y in zip(self.D.iterrows(), self.Y):
            if y == self.POS:
                self.handle_positive(d)
            elif y == self.NEG:
                self.handle_negative(d)
            else:
                raise InvalidClassError

            print()
            print(f"G in {i+1} iterations:", self.G)
            print(f"S in {i+1} iterations:", self.S)
            print()

    def handle_positive(self, d: pd.Series):
        print("handling positive...")
        # remove inconsistent hypos in G
        for g in self.G:
            if not g.is_consistent_with(d):
                self.G.remove(g)

        # generalize S
        for s in self.S:
            if not s.is_consistent_with(d):
                # TODO: add all generalizations
                generalized_s = genenalize(s)
                
                has_more_general_in_G = false
                for g in self.G:
                    if g.is_more_general_than(s):
                        has_more_general_in_G = true
                        break
    
                if generalized_s.is_consistent_with(d) and \
                    has_more_general_in_G:
                    self.S.remove(s)
                    self.S.append(generalized_s)

        # remove more generals in S
        purged_S: List[Hypothesis] = []
        for i in range(len(self.S)):
            ok = True
            for j in range(len(self.S)):
                if i == j:
                    continue

                if self.S[i].is_more_general_than(self.S[j]):
                    ok = False
                    break

            if ok:
               purged_S.append(self.S[i]) 

        self.S = purged_S

        
    def handle_negative(self, d: pd.Series):
        print("handling negative...")
        # remove inconsistent hypos in S
        for s in self.S:
            if not s.is_consistent_with(d):
                self.S.remove(s)

        # specialize G
        for g in self.G:
            if not g.is_consistent_with(d):
                # TODO: add all specializations
                specialized_g = specialize(g)

                has_more_specific_in_S = false
                for s in self.S:
                    if s.is_more_specific_than(g):
                        has_more_specific_in_S = true
                        break

                if specialized_g.is_consistent_with(d) and \
                    has_more_specific_in_S:
                    self.G.remove(g)
                    self.G.append(specialized_g)

        # remove more specials in G
        purged_G: List[Hypothesis] = []
        for i in range(len(self.G)):
            ok = True
            for j in range(len(self.G)):
                if i == j:
                    continue

                if self.G[i].is_more_specific_than(self.G[j]):
                    ok = False
                    break

            if ok:
                purged_G.append(self.G[i])

        self.G = purged_G
                
def generalize(h: Hypothesis) -> Hypothesis:
    return h


ce = CandidateElimination()
X = df.drop("class", axis=1)
Y = df["class"]

ce.fit(X.head(1), Y, 1, 0)
print("final G:", ce.G)
print("final S:", ce.S)

initial G: [['?', '?', '?', '?', '?', '?']]
initial S: [['0', '0', '0', '0', '0', '0']]
handling negative...

G in 1 iterations: [['?', '?', '?', '?', '?', '?']]
S in 1 iterations: []

final G: [['?', '?', '?', '?', '?', '?']]
final S: []


<hr>

<b>List-Then-Eliminate</b>


- $VersionSpace$ $\rightarrow$ a list containing every hypothesis in $H$
- For each training example $<x, c(x)>$:
    - remove from $VersionSpace$ any hypothesis $h$ for which $h(x) \neq c(x)$

In [28]:
from itertools import product

class ListThenEliminate:
    def __init__(self) -> None:
        pass

    def fit(self, D: pd.DataFrame, Y: pd.Series, pos: int = 1, neg: int = 0) -> None:
        self.D = D
        self.Y = Y
        self.POS = pos
        self.NEG = neg
        self.VS = calculate_entire_version_space(self.D)

        print(f"starting with {len(self.VS)} hypotheses...")

        idx = 1
        for i, d in self.D.iterrows():
            c_x = Y[i]
            for h in self.VS:
                h_x = h.predict(d)
                if c_x != h_x:
                    self.VS.remove(h)
                    idx += 1
    
def calculate_entire_version_space(D: pd.DataFrame) -> List[Hypothesis]:
    vs: List[Hypothesis] = [Hypothesis([NULL for _ in D.columns])]
    column_values = [set(D[column].tolist() + [ANY]) for column in D.columns]
    for h in list(set(product(*column_values))):
        vs.append(Hypothesis(h))
        
    return vs

lte = ListThenEliminate()

X = df.drop("class", axis=1)
Y = df["class"]
lte.fit(X, Y)

print("final version space:")
for v in lte.VS:
    print(v)

starting with 973 hypotheses...
final version space:
['?', 'Warm', '?', '?', '?', '?']
['?', 'Warm', '?', 'Weak', '?', '?']
['Sunny', '?', 'Normal', '?', '?', '?']
['?', 'Warm', '?', '?', 'Warm', '?']
['Sunny', 'Warm', '?', '?', '?', '?']
['?', 'Warm', 'Normal', '?', 'Warm', '?']
['Sunny', 'Warm', 'Normal', 'Weak', '?', '?']
['?', 'Warm', 'Normal', '?', '?', '?']
['?', 'Warm', '?', 'Weak', 'Warm', '?']
['?', 'Warm', 'Normal', 'Weak', '?', '?']
